In [104]:
import pandas as pd
import numpy as np
import math

#### Read Data

In [105]:
df = pd.read_csv('D:\FirstSemMT\ML\Assignments\leukemia.tab',sep='\t')
df.shape

(75, 5148)

In [106]:
df.head()

,gene,AFFX-BioC-5_at,hum_alu_at,AFFX-DapX-M_at,AFFX-LysX-5_at,AFFX-HUMISGF3A/M97935_MA_at,AFFX-HUMISGF3A/M97935_MB_at,AFFX-HUMISGF3A/M97935_3_at,AFFX-HUMRGE/M10098_5_at,AFFX-HUMRGE/M10098_M_at,...,M93143_at,U29175_at,U48730_at,U58516_at,X06956_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
0,discrete,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous
1,class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALL,88,15091,311,21,-13,215,797,14538,9738,...,384,1582,185,511,389,793,329,36,191,-37
3,ALL,283,11038,134,-21,-219,116,433,615,115,...,231,624,169,837,442,782,295,11,76,-14
4,ALL,309,16692,378,67,104,476,1474,5669,3272,...,720,753,315,1199,168,1138,777,41,228,-41


In [107]:
df.drop(df.index[df['gene'] == 'discrete'], inplace = True)
df.drop(df.index[df['gene'] == 'class'], inplace = True)
df.drop(df.index[72], inplace = True)
label = df['gene'] 
df.drop(['gene'], axis = 1, inplace = True)
df = df.values
samples,features = df.shape
print(samples,features)

72 5147


In [108]:
data = df.astype(np.float)
data.shape

(72, 5147)

#### Import train_test_split function and Split dataset into training set and test set

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2,random_state=100) 

#### Find Mutual Info for every feature 

In [110]:
ig = np.zeros(features)

c1 = np.where(y_train=='ALL')[0]
c2 = np.where(y_train=='AML')[0]

u = np.array([c1.shape[0],c2.shape[0]])
ans1=0
ic = 0
for p in range(2):
    ans1 = ans1 - ((u[p]/u.sum())*math.log2(u[p]/(u.sum())))
ic = ans1            

for i in range(features): 
    minElement = np.amin(X_train[:,i])
    maxElement = np.amax(X_train[:,i])
    diff = (maxElement-minElement)/2
    
    a = minElement
    b = a+diff
    c = a+(2*diff)
    
    x = np.where(np.logical_and(X_train[:,i]>=a,X_train[:,i]<b))[0] 
    y = np.where(np.logical_and(X_train[:,i]>=b,X_train[:,i]<c))[0] 
    
    xc1=np.intersect1d(x,c1)
    yc1=np.intersect1d(y,c1)
    xc2=np.intersect1d(x,c2)
    yc2=np.intersect1d(y,c2)
    
    v = np.array([[xc1.shape[0],yc1.shape[0]],[xc2.shape[0],yc2.shape[0]]])
    ans2=0
    for p in range(2):
        for q in range(2):
            if(v[q][p]!=0):
                ans2 = ans2 - ((v[q][p]/v.sum())*math.log2(v[q][p]/(v[0][p]+v[1][p])))
    ig[i] = ic-ans2
print(ig)
    

[0.01997942 0.03810999 0.00621657 ... 0.00283634 0.05001549 0.05493751]


#### Sort indexes in descending order of mutual info

In [111]:
sort_idxs=(-ig).argsort()[:ig.shape[0]]
sort_idxs

array([2423, 1223, 1548, ..., 4310,  204, 1768], dtype=int64)

#### Select top features (less than 20%)

In [112]:
filt_idxs = sort_idxs[0:int((data.shape[1])*0.2)]

In [113]:
X_new_train = X_train[:,filt_idxs]
X_new_test = X_test[:,filt_idxs]
X_new_train.shape

(57, 1029)

#### KNN Classification

In [114]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_new_train, y_train)
y_pred = knn.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using KNN

In [115]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9333333333333333
f1 score: 0.9206349206349207
confusion matrix: [[10  0]
 [ 1  4]]


#### SNM Classification

In [116]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_new_train, y_train)
y_pred = clf.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using SVM

In [117]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9333333333333333
f1 score: 0.9282296650717703
confusion matrix: [[9 1]
 [0 5]]
